In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import pooch

from fair import FAIR
from fair.io import read_properties
from fair.interface import initialise, fill

In [ ]:
f = FAIR()

In [ ]:
f.define_time(1750, 2100, 1)

In [ ]:
f.define_scenarios(['ssp126'])

In [ ]:
pd.options.display.max_columns = 50
fair_params_df = pd.read_csv('../data/fair2.1-parameters/ar6_calibration_ebm3.csv', index_col=0)
fair_params_df

In [ ]:
species, properties = read_properties()

In [ ]:
f.define_species(species, properties)

In [ ]:
f.define_configs(list(fair_params_df.index))

In [ ]:
f.allocate()

In [ ]:
f.fill_from_rcmip()

In [ ]:
ar6_forcing_file = pooch.retrieve(
    url="doi:10.5281/zenodo.5705391/table_A3.3_historical_ERF_1750-2019_best_estimate.csv",
    known_hash="md5:f64915777d2971c5eb5d96a432f45c48",
)

In [ ]:
ar6_forcing_df = pd.read_csv(ar6_forcing_file, index_col=0)

In [ ]:
volcanic_forcing = np.zeros(351)
volcanic_forcing[:270] = ar6_forcing_df['volcanic'].values
volcanic_forcing[269:281] = np.linspace(1, 0, 12) * volcanic_forcing[269]

In [ ]:
solar_forcing=np.zeros(351)
solar_forcing_df = pd.read_csv('../data/forcing/solar_erf.csv', index_col=0)
solar_forcing = solar_forcing_df['solar_erf'].values[:351]

In [ ]:
# Volcanic forcing
fill(f.forcing, volcanic_forcing[:, None, None] * fair_params_df.loc[:, 'scale Volcanic'].values[None, None, :], specie='Volcanic')

# Solar forcing
trend_shape = np.ones(351)
trend_shape[:271] = np.linspace(0, 1, 271)

fill(f.forcing, 
     solar_forcing[:, None, None] * 
     fair_params_df.loc[:, 'solar_amplitude'].values.squeeze() + 
     trend_shape[:, None, None] * fair_params_df.loc[:, 'solar_trend'].values.squeeze(),
     specie='Solar'
)

In [ ]:
f.fill_species_configs()

In [ ]:
# Climate response
fill(f.climate_configs['ocean_heat_capacity'], fair_params_df.loc[:,'c1':'c3'])
fill(f.climate_configs['ocean_heat_transfer'], fair_params_df.loc[:,'kappa1':'kappa3'])
fill(f.climate_configs['deep_ocean_efficacy'], fair_params_df.loc[:,'epsilon'])
fill(f.climate_configs['sigma_eta'], fair_params_df.loc[:,'sigma_eta'])
fill(f.climate_configs['sigma_xi'], fair_params_df.loc[:,'sigma_xi'])
fill(f.climate_configs['gamma_autocorrelation'], fair_params_df.loc[:,'gamma'])
fill(f.climate_configs['seed'], fair_params_df.index*399 + 1355763)
fill(f.climate_configs['use_seed'], True)
fill(f.climate_configs['stochastic_run'], True)

# carbon cycle
fill(f.species_configs['iirf_0'], fair_params_df.loc[:, 'r0'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_airborne'], fair_params_df.loc[:, 'rA'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_uptake'], fair_params_df.loc[:, 'rU'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_temperature'], fair_params_df.loc[:, 'rT'].values.squeeze(), specie='CO2')

# aerosol direct
for specie in ['BC', 'CH4', 'N2O', 'NH3', 'NOx', 'OC', 'Sulfur', 'VOC', 'Equivalent effective stratospheric chlorine']:
    fill(f.species_configs['erfari_radiative_efficiency'], fair_params_df.loc[:, f'ari {specie}'].values.squeeze(), specie=specie)

# aerosol indirect
fill(f.species_configs['aci_parameters'], fair_params_df.loc[:, 'beta'].values.squeeze(), aci_parameter='scale')
fill(f.species_configs['aci_parameters'], fair_params_df.loc[:, 'shape_so2'].values.squeeze(), aci_parameter='Sulfur')
fill(f.species_configs['aci_parameters'], fair_params_df.loc[:, 'shape_bcoc'].values.squeeze(), aci_parameter='BC+OC')

# ozone
for specie in ['CH4', 'N2O', 'Equivalent effective stratospheric chlorine', 'CO', 'VOC', 'NOx']:
    fill(f.species_configs['ozone_radiative_efficiency'], fair_params_df.loc[:, f'o3 {specie}'], specie=specie)

# methane lifetime baseline
fill(f.species_configs['unperturbed_lifetime'], 10.4198121, specie='CH4')

# emissions adjustments for N2O and CH4 (we don't want to make these defaults as people might wanna run pulse expts with these gases)
fill(f.species_configs['baseline_emissions'], 19.019783117809567, specie='CH4')
fill(f.species_configs['baseline_emissions'], 0.08602230754, specie='N2O')

# forcing scaling

for specie in ['CH4', 'N2O', 'Stratospheric water vapour', 'Contrails', 'Light absorbing particles on snow and ice', 'Land use']:
    fill(f.species_configs['forcing_scale'], fair_params_df.loc[:, f'scale {specie}'].values.squeeze(), specie=specie)
for specie in ['CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115', 'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br', 'Halon-1211', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C3F8', 'c-C4F8', 'C4F10', 'C5F12', 'C6F14', 'C7F16', 'C8F18', 'NF3', 'SF6', 'SO2F2',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32',
    'HFC-365mfc', 'HFC-4310mee']:
    fill(f.species_configs['forcing_scale'], fair_params_df.loc[:, 'scale minorGHG'].values.squeeze(), specie=specie)

calibrated_f4co2_mean = fair_params_df.loc[:,'F_4xCO2'].values.mean()
fill(f.species_configs['forcing_scale'], 1 + 0.561*(calibrated_f4co2_mean - fair_params_df.loc[:,'F_4xCO2'].values)/calibrated_f4co2_mean, specie='CO2')


# tune down volcanic efficacy
fill(f.species_configs['forcing_efficacy'], 0.6, specie='Volcanic')


# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(f.species_configs['baseline_concentration'], fair_params_df.loc[:, 'co2_concentration_1750'].values.squeeze(), specie='CO2')

f.ch4_method='Thornhill2021'

In [ ]:
initialise(f.concentration, f.species_configs['baseline_concentration'])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.airborne_emissions, 0)
initialise(f.cumulative_emissions, 0)

In [ ]:
f.run()

In [ ]:
pl.plot(f.timebounds, f.temperature[:, 0, :, 0] - f.temperature[100:151, 0, :, 0].mean(axis=0));
pl.xlim(1850, 2100)
pl.ylim(-.5, 3.5)

In [ ]:
pl.plot(f.timebounds, f.temperature[:, 0, 0, 0] - f.temperature[100:151, 0, 0, 0].mean(axis=0));
pl.axhline(1.5, color='r')
pl.xlim(1850, 2100)
pl.ylim(-.5, 3.5)

In [ ]:
calibrated_f4co2_mean